# Import Python Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Approach
In this notebook, revenue will be estimated with an ensemble of machine learning techniques. Specifically:
* Because there are many predictors, the model space will be managed by
    * PCA to compress variable space and/or
    * Ridge Regression to minimize overfitting by large coefficient values 
* Regression 
* Regression Trees

# Read in Data
Some initial preprocessing was done:
* Replaced blanks with NAs (NaN in df below)
* Moved the reponse variable (revenue) to the first field

Preprocessing done in R ('nuggetsPreprocess.R')
* Changed binary variables to y/n for portability
* Added variables diff1 - diff4 and weekOfYear for handling temporal autocorrelation and seasonality

In [13]:
df = pd.read_csv('dataClean.csv')
df2 = pd.read_csv('dataCleanNoOutlier.csv')
df.head()

,revenue,Week,weekno,highseason,holiday,displayimp,displaylag,brandpaid,brandclicks,nonbrandpaid,...,blogsignup,autoenroll,socialperson,promoteposts,learner,diff1,diff2,diff3,diff4,weekOfYear
0,178480,2/2/14,1,y,n,987718,983958.0000,31589,4715,105901,...,18,y,y,n,n,NaN,NaN,NaN,NaN,5
1,185390,2/9/14,2,y,n,660136,907388.9167,30428,4540,108652,...,16,y,y,n,n,6910,NaN,NaN,NaN,6
2,178130,2/16/14,3,y,n,604606,852862.4167,30715,4417,119398,...,21,y,y,n,n,-7260,-14170,NaN,NaN,7
3,185020,2/23/14,4,y,n,560656,851885.3333,31632,4372,134819,...,19,y,y,n,n,6890,14150,28320,NaN,8
4,170950,3/2/14,5,y,n,326112,827164.4167,NaN,4431,115865,...,11,y,y,n,n,-14070,-20960,-35110,-63430,9


## Parse Data into Train and Test Sets
Because we are looking at two data sets (with and without "outliers" removed), we want the same data to be in each Test and Training set for direct comparison.

In [11]:
df.shape

(117, 47)